Import and setup some auxiliary functions

In [ ]:
import torch
from torchvision import datasets, transforms
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat


def compute_score(acc, min_thres, max_thres):
    if acc <= min_thres:
        base_score = 0.0
    elif acc >= max_thres:
        base_score = 100.0
    else:
        base_score = float(acc - min_thres) / (max_thres - min_thres) \
                     * 100
    return base_score


def run(algorithm, x_train, y_train, x_test, y_test, n_classes, device,k):
    print('Running...')        
    if device != 'cpu' and torch.cuda.is_available():
        device = torch.device("cuda")
        print('Training on GPU: {}'.format(torch.cuda.get_device_name(0)))
    else:
        device = torch.device("cpu")
        print('Training on CPU')
        
    start = timeit.default_timer()
    np.random.seed(0)
    predicted_y_test = algorithm(x_train, y_train, x_test, n_classes, device,k)
    np.random.seed()
    stop = timeit.default_timer()
    run_time = stop - start

    correct_predict = (y_test
                       == predicted_y_test).astype(np.int32).sum()
    incorrect_predict = len(y_test) - correct_predict
    accuracy = float(correct_predict) / len(y_test)

    print('Correct Predict: {}/{} total \tAccuracy: {:5f} \tTime: {:2f} k: {:n}'.format(correct_predict,
                                                                                len(y_test), accuracy, run_time, k))
    return correct_predict, accuracy, run_time

TODO: Implement knn here

In [ ]:
def knn(x_train, y_train, x_test, n_classes, device,k):
    """
    x_train: 60000 x 784 matrix: each row is a flattened image of an MNIST digit
    y_train: 60000 vector: label for x_train
    x_test: 1000 x 784 testing images
    n_classes: no. of classes in the classification task
    device: pytorch device on which to run the code
    return: predicted y_test which is a 1000-sized vector
    """
    """
    create a numpy array of zeros for predictions
    dtype is automatically recognized by analysing y_train for classification purpose
    """
    ypred = np.zeros(1000,dtype = y_train.dtype)

    """ 
    Convert training and test arrays to tensor ; reference = https://pytorch.org/docs/stable/tensors.html
    dtypes are set to float as well as the device is automatically recognized
    """
    x_train= torch.tensor(x_train, dtype=torch.float, device=device)
    x_test = torch.tensor(x_test, dtype=torch.float, device=device)

    """ 
    loop over the range of 1000 testing images
    """
    for index in range(x_test.shape[0]):

      """ 
      find the L2 norm of the training set - test set (here p = 2 means the L2 norm)
      After hyper paramenter tuning the best value of k was found to be 1
      the p value was found to be 4 which yielded a best accuracy of 97% although slower which yielded in a loss of 2 seconds
      the p value = 2 that is the euclidean distance yielded 96.3% in 4.4 at best and on average at 5 seconds.
      """
      distances = torch.norm(x_train-x_test[index],p = 4,dim = 1)

      """
      find the min element and the index it is at ; reference = https://pytorch.org/docs/stable/generated/torch.topk.html
      largest is set to false as we want the smallest element and not the largest one
      """
      element,min_index = torch.topk(distances,1,largest = False)
  
      """
      find the value of it in the y_train set and assign it to predictions
      """
      ypred[index] = y_train[min_index]
      
    return ypred

Main loop. You can only run this after filling the knn function above

In [ ]:
min_thres = 0.84
max_thres = 0.94
n_classes = 10
# change to cpu to run on CPU
device = 'gpu'

mnist_train = datasets.MNIST('data', train=True, download=True,
                             transform=transforms.Compose([
                                 transforms.Normalize((0.1307,), (0.3081,)),
                             ])
                             )
mnist_test = datasets.MNIST('data', train=False, download=True,
                             transform=transforms.Compose([
                                 transforms.Normalize((0.1307,), (0.3081,)),
                             ])
                            )
result = [OrderedDict(first_name='Robert',last_name='Joseph')]

# convert pytorch tensors to numpy arrays
(x_train, y_train) = (mnist_train.data.cpu().numpy(), mnist_train.targets.cpu().numpy())
(x_valid, y_valid) = (mnist_test.data.cpu().numpy(), mnist_test.targets.cpu().numpy())

# flatten 28x28 images into 784 sized vectors
x_train = x_train.reshape(x_train.shape[0], -1)
x_valid = x_valid.reshape(x_valid.shape[0], -1)

# You may want to use a smaller training set to save time when debugging
# i.e.: Put something like:
#(x_train, y_train) = (x_train[:5000], y_train[:5000])

# For this assignment, we only test on the first 1000 samples of the test set
(x_valid, y_valid) = (x_valid[:1000], y_valid[:1000])

#z = knn(x_train,y_train,x_valid,n_classes,device)
#print(z)

print("Dimension of dataset: ")
print("Train:", x_train.shape, y_train.shape, "\nTest:", x_valid.shape, y_valid.shape)


(correct_predict, accuracy, run_time) = run(knn, x_train, y_train, x_valid, y_valid, n_classes, device,1)
score = compute_score(accuracy, min_thres, max_thres)
result = OrderedDict(correct_predict=correct_predict,accuracy=accuracy, score=score,run_time=run_time)

with open('result.txt', 'w') as f:
    f.writelines(pformat(result, indent=4))

print(pformat(result, indent=4))

Dimension of dataset: 
Train: (60000, 784) (60000,) 
Test: (1000, 784) (1000,)
Running...
Training on GPU: Tesla T4
Correct Predict: 967/1000 total 	Accuracy: 0.967000 	Time: 3.430022 k: 1
OrderedDict([   ('correct_predict', 967),
                ('accuracy', 0.967),
                ('score', 100.0),
                ('run_time', 3.4300224259999936)])
